In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/updated-data/hello.csv
/kaggle/input/bert-base-multilingual-epoch-18/bert-base-multilingual-uncased-False-18.pt
/kaggle/input/evalsem24/MaSaC_test_erc.json
/kaggle/input/mbert-l3-cube-epoch-9/hing-mbert-False-9.pt


In [2]:
!pip install ml_collections wandb
!wandb login dedc08f0b13e84705452beca60fc2b5f46430ff9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 812.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for ml_collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94506 sha256=6d59328faa890d851a6eb334ddd46e71cf588cecd8a7ba854da9880b27b830b3
  Stored in directory: /root/.cache/pip/wheels/7b/89/c9/a9b87790789e94aadcfc393c283e3ecd5ab916aed0a31be8fe
Successfully built ml_collections
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import torch
import wandb
import warnings
import random
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from datetime import datetime
from ml_collections import ConfigDict
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
from sklearn.metrics import classification_report,f1_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from ml_collections import ConfigDict
from torch.utils.data import DataLoader, Dataset

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

cfg = ConfigDict()
cfg.model_name = "bert-base-multilingual-uncased"
emotion_mapping = {
    "disgust": 0,
    "contempt": 1,
    "anger": 2,
    "neutral": 3,
    "joy": 4,
    "sadness": 5,
    "fear": 6,
    "surprise": 7,
}

class DialogueDataset(Dataset):
    def __init__(self, test_data_path):
        super().__init__()
        self.dataframe = pd.read_csv(test_data_path)
        self.label_mappings = emotion_mapping

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        utterance = self.dataframe["utterances"].to_list()
        return utterance[index]

    def create_loader(self, batch_size):
        return DataLoader(self, batch_size=batch_size, shuffle=False)

class SemEvalNet(torch.nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.backbone = AutoModel.from_pretrained(model_name)
        self.linear_layer = torch.nn.Linear(768, self.num_classes)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.push_to_device()

    def forward(self, batch):
        x = self.backbone(**batch).pooler_output
        x = self.linear_layer(x)
        return x

    def push_to_device(self):
        self.backbone.to(self.device)
        self.linear_layer.to(self.device)

    def predict(self, test_loader):
        predictions = []
        self.eval()
        with torch.no_grad():
            for batch in tqdm(test_loader):
                inputs = self.tokenizer(
                   text=list(batch),
                   return_attention_mask=True,
                   max_length=256,
                   padding="max_length",
                   truncation=True,
                   return_tensors="pt",
                )
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                scores = self(inputs)
                predictions.extend(scores.argmax(dim=-1).cpu().numpy())
        return predictions



In [5]:
# Test data path
test_data_path = "/kaggle/input/updated-data/hello.csv"

# Load test dataset
test_dataset = DialogueDataset(test_data_path)
test_loader = test_dataset.create_loader(batch_size=8)

# Model instantiation
model = SemEvalNet(model_name=cfg.model_name, num_classes=8)
model.load_state_dict(torch.load("/kaggle/input/bert-base-multilingual-epoch-18/bert-base-multilingual-uncased-False-18.pt"))  # Replace with the path to your model

emotion_mapping_inv = {v: k for k, v in emotion_mapping.items()}
# Make predictions
predicted_labels = model.predict(test_loader)

# Map predicted labels to emotions
predicted_emotions = [emotion_mapping_inv[label] for label in predicted_labels]

# Save predictions to a CSV file
predictions_df = pd.DataFrame(predicted_emotions)
predictions_df.to_csv("predicted_emotions.csv", index=False)
#{"utterances": test_dataset.dataframe["utterances"],}

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

100%|██████████| 198/198 [00:22<00:00,  8.88it/s]


In [6]:
print("Length of utterances:", len(test_dataset.dataframe["utterances"]))
print("Length of predicted_emotions:", len(predicted_emotions))

Length of utterances: 1580
Length of predicted_emotions: 1580


In [7]:
df2=pd.read_csv("/kaggle/working/predicted_emotions.csv")

In [8]:
df2

,0
0,sadness
1,fear
2,surprise
3,neutral
4,anger
...,...
1575,neutral
1576,neutral
1577,joy
1578,neutral


In [9]:
#df2

In [10]:
#df1=pd.read_csv(test_data_path)

In [11]:
#df1

In [12]:
#df3=pd.read_csv("/kaggle/working/predicted_emotions_batch16.csv")

In [13]:
#df3.shape

In [14]:
#df4=pd.read_csv("/kaggle/working/predicted_emotions_batch32.csv")

In [15]:
#df4.shape

In [16]:
#df5=pd.read_csv("/kaggle/working/predicted_emotions_batch64.csv")

In [17]:
#df5.shape